In [ ]:
#VS2Net_data

In [ ]:
#step1

In [ ]:
import pandas as pd
from scipy.special import wofz
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
from tqdm import trange
import seaborn as sns
from rdkit import Chem
from rdkit.Chem.rdmolops import GetAdjacencyMatrix
import torch
from torch_geometric.data import Data
from torch.utils.data import DataLoader

In [ ]:
def one_hot_encoding(x, permitted_list):
    """
    Maps input elements x which are not in the permitted list to the last element
    of the permitted list.
    """

    if x not in permitted_list:
        x = permitted_list[-1]

    binary_encoding = [int(boolean_value) for boolean_value in list(map(lambda s: x == s, permitted_list))]

    return binary_encoding
def get_atom_features(atom,
                      use_chirality = True,
                      hydrogens_implicit = True):
    #输入rdkit atom，输出1d numpy array
    permitted_list_of_atoms =  ['C','N','O','S','F','P','Cl','Br','Unknown']
    if hydrogens_implicit == False:
        permitted_list_of_atoms = ['H'] + permitted_list_of_atoms
        
    atom_type_enc = one_hot_encoding(str(atom.GetSymbol()),permitted_list_of_atoms)
    n_heavy_neighbors_enc = one_hot_encoding(int(atom.GetDegree()), [0, 1, 2, 3, 4, "MoreThanFour"])
    formal_charge_enc = one_hot_encoding(int(atom.GetFormalCharge()), [-3, -2, -1, 0, 1, 2, 3, "Extreme"])
    hybridisation_type_enc = one_hot_encoding(str(atom.GetHybridization()), ["S", "SP", "SP2", "SP3", "SP3D", "SP3D2", "OTHER"])
    is_in_a_ring_enc = [int(atom.IsInRing())]
    is_aromatic_enc = [int(atom.GetIsAromatic())]
    atomic_mass_scaled = [float((atom.GetMass() - 10.812)/116.092)]
    vdw_radius_scaled = [float((Chem.GetPeriodicTable().GetRvdw(atom.GetAtomicNum()) - 1.5)/0.6)]
    covalent_radius_scaled = [float((Chem.GetPeriodicTable().GetRcovalent(atom.GetAtomicNum()) - 0.64)/0.76)]
    
    atom_feature_vector = atom_type_enc + n_heavy_neighbors_enc + formal_charge_enc + hybridisation_type_enc + is_in_a_ring_enc + is_aromatic_enc + atomic_mass_scaled + vdw_radius_scaled + covalent_radius_scaled
    
    if use_chirality == True:
        chirality_type_enc = one_hot_encoding(str(atom.GetChiralTag()), ["CHI_UNSPECIFIED", "CHI_TETRAHEDRAL_CW", "CHI_TETRAHEDRAL_CCW", "CHI_OTHER"])
        atom_feature_vector += chirality_type_enc
    if hydrogens_implicit == True:
        n_hydrogens_enc = one_hot_encoding(int(atom.GetTotalNumHs()), [0, 1, 2, 3, 4, "MoreThanFour"])
        atom_feature_vector += n_hydrogens_enc
        
    return np.array(atom_feature_vector)


def get_bond_features(bond, 
                      use_stereochemistry = True):
   #输入rdkit bond，输出1d numpy array

    permitted_list_of_bond_types = [Chem.rdchem.BondType.SINGLE, Chem.rdchem.BondType.DOUBLE, Chem.rdchem.BondType.TRIPLE, Chem.rdchem.BondType.AROMATIC]

    bond_type_enc = one_hot_encoding(bond.GetBondType(), permitted_list_of_bond_types)    
    bond_is_conj_enc = [int(bond.GetIsConjugated())]    
    bond_is_in_ring_enc = [int(bond.IsInRing())]  
    
    bond_feature_vector = bond_type_enc + bond_is_conj_enc + bond_is_in_ring_enc
    
    if use_stereochemistry == True:
        stereo_type_enc = one_hot_encoding(str(bond.GetStereo()), ["STEREOZ", "STEREOE", "STEREOANY", "STEREONONE"])
        bond_feature_vector += stereo_type_enc

    return np.array(bond_feature_vector)

In [ ]:
#target,mode = extract_labels(all_files[i])

def get_feature_and_labels(ii,target,mode):
    #对于x---输入图（atom数*45维）edges_index(2*键数)edge_attr(键数*10)
    #对于y---输出nparray50维
    smi = data.iloc[ii][0]
    mol = Chem.MolFromSmiles(data.iloc[ii][0])
    target = target
    mode = mode

    n_nodes = mol.GetNumAtoms()
    n_edges = 2*mol.GetNumBonds()
    X =np.zeros((n_nodes,45))
    
    for atom in mol.GetAtoms():
        X[atom.GetIdx(),:] = get_atom_features(atom)
    X = torch.tensor(X,dtype = torch.float)

    # construct edge index array E of shape (2, n_edges)
    (rows, cols) = np.nonzero(GetAdjacencyMatrix(mol))
    torch_rows = torch.from_numpy(rows.astype(np.int64)).to(torch.long)
    torch_cols = torch.from_numpy(cols.astype(np.int64)).to(torch.long)
    E = torch.stack([torch_rows, torch_cols], dim = 0)
    
    # construct edge feature array EF of shape (n_edges, n_edge_features)
    EF = np.zeros((n_edges, 10))        
    for (k, (m,j)) in enumerate(zip(rows, cols)):
        EF[k] = get_bond_features(mol.GetBondBetweenAtoms(int(m),int(j)))        
    EF = torch.tensor(EF, dtype = torch.float)
    
    #y_tensor = torch.tensor(y_50_tran(i))
    return Data(x=X,edge_index=E,edge_attr=EF,target = target,mode=mode,smi=smi)

In [ ]:
all_files = []
root_dir = "/home/chengc/workspace/cc/data/dude/dude_smi"
for dirpath, dirnames, files in os.walk(root_dir):  
    for name in files:  
        all_files.append(os.path.join(dirpath, name)) 
def extract_labels(file_path):  
    # 分割文件名和目录部分  
    filename = os.path.basename(file_path)  
    # 去掉文件扩展名  
    filename_without_ext = os.path.splitext(filename)[0]  
    # 分割文件名以获得前缀和标签  
    parts = filename_without_ext.split('_')  
    # 假设前缀和标签分别位于倒数第二和最后一个位置  
    target_protein = parts[-4] if len(parts) > 1 else None  
    mode = parts[-3] if len(parts) > 0 else None  
    # 返回前缀和标签  
    return target_protein, mode

In [ ]:
for i in trange(len(all_files)):
    target,mode = extract_labels(all_files[i])
    data = pd.read_csv(all_files[i])
    data_all = []
    save_dir = os.path.join("/home/chengc/workspace/cc/0815/data/cal_data/dude_graph",target+"_"+mode+".pt")
    for ii in range(len(data)):
        point = get_feature_and_labels(ii,target,mode)
        data_all.append(point)
    torch.save(data_all,save_dir) 

In [ ]:
#step2

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
import torch.nn as nn
from torch_geometric.nn import GCNConv,EdgeConv
from torch_geometric.nn import global_mean_pool,global_max_pool
from torch.utils.data import random_split
import torch.nn.functional as F
from tqdm import trange

In [ ]:
###模型导入
class GNN_COS(nn.Module):
    
    def __init__(self):
        super().__init__()
        torch.manual_seed(42)
       
        self.conv1 = GCNConv(45, hidden_channels)
        self.conv2 = GCNConv(hidden_channels,hidden_channels)
        self.conv3 = GCNConv(hidden_channels,hidden_channels)
        self.conv4 = GCNConv(hidden_channels,hidden_channels)
        self.conv5 = GCNConv(hidden_channels,hidden_channels)
        
        self.lin1 = nn.Linear(2*hidden_channels,hidden_channels)
        self.lin2 = nn.Linear(hidden_channels,hidden_channels)
        self.out = nn.Linear(hidden_channels, 50)
        
        self.loss_function1 = nn.CosineEmbeddingLoss(margin=0)
        self.loss_function2 = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.0001, weight_decay=5e-4)
        self.counter=0
        self.progress = []
        
    def forward(self, x, edge_index, batch):
        
        x = x.to(device)
        edge_index = edge_index.to(device)
        batch = batch.to(device)
        
        x=self.conv1(x, edge_index)
        x = x.relu()
        x=self.conv2(x, edge_index)
        x = x.relu()
        x=self.conv3(x, edge_index)
        x = x.relu()
        x=self.conv4(x, edge_index)
        x = x.relu()
        x=self.conv5(x, edge_index)
        x = x.relu()
        
        x = torch.cat((global_mean_pool(x,batch),global_max_pool(x,batch)),dim=1)
        x = self.lin1(x)
        x = x.relu()
        x = self.lin2(x)
        x = x.relu()
        out = self.out(x)
        return out
    
    def train(self,data):
        
        self.optimizer.zero_grad()
        
        outputs = self.forward(data.x.float(),data.edge_index,data.batch)
        y = data.y.to(device)
        batch_size = data.batch_size
        target = torch.ones(batch_size)
        target = target.to(device)
        #loss = cos_ratio*self.loss_function1(outputs.to(torch.float32),y.view(batch_size,50).to(torch.float32),target)+mse_ratio*self.loss_function2(outputs.to(torch.float32),y.view(batch_size,50).to(torch.float32))
        loss = self.loss_function1(outputs.to(torch.float32),y.view(batch_size,50).to(torch.float32),target)
        self.progress.append(loss.item())        
        loss.backward()
        self.optimizer.step()
        
        return loss
        
    #def test(self,data):
        #outputs = self.forward(data.x.float(),data.edge_index,data.batch)
        #target = torch.ones(len(data))
        #target = target.to(device)
        #y = data.y.to(device)
        #acc = cos_ratio*self.loss_function1(outputs.to(torch.float32),y.view(len(test_data),50).to(torch.float32),target)+mse_ratio*self.loss_function2(outputs.to(torch.float32),y.view(len(test_data),50).to(torch.float32))
        #acc = self.loss_function1(outputs.to(torch.float32),y.view(len(data),50).to(torch.float32),target)
        #return acc
    
    def pred(self,data):
        outputs = self.forward(data.x.float(),data.edge_index,data.batch)
        return outputs



class GNN_MSE(nn.Module):
    
    def __init__(self):
        super().__init__()
        torch.manual_seed(42)
       
        self.conv1 = GCNConv(45, hidden_channels)
        self.conv2 = GCNConv(hidden_channels,hidden_channels)
        self.conv3 = GCNConv(hidden_channels,hidden_channels)
        self.conv4 = GCNConv(hidden_channels,hidden_channels)
        self.conv5 = GCNConv(hidden_channels,hidden_channels)
        
        self.lin1 = nn.Linear(2*hidden_channels,hidden_channels)
        self.lin2 = nn.Linear(hidden_channels,hidden_channels)
        self.out = nn.Linear(hidden_channels, 50)
        
        self.loss_function1 = nn.CosineEmbeddingLoss(margin=0)
        self.loss_function2 = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.0001, weight_decay=5e-4)
        self.counter=0
        self.progress = []
        
    def forward(self, x, edge_index, batch):
        
        x = x.to(device)
        edge_index = edge_index.to(device)
        batch = batch.to(device)
        
        x=self.conv1(x, edge_index)
        x = x.relu()
        x=self.conv2(x, edge_index)
        x = x.relu()
        x=self.conv3(x, edge_index)
        x = x.relu()
        x=self.conv4(x, edge_index)
        x = x.relu()
        x=self.conv5(x, edge_index)
        x = x.relu()
        
        x = torch.cat((global_mean_pool(x,batch),global_max_pool(x,batch)),dim=1)
        x = self.lin1(x)
        x = x.relu()
        x = self.lin2(x)
        x = x.relu()
        out = self.out(x)
        return out
    
    def train(self,data):
        
        self.optimizer.zero_grad()
        
        outputs = self.forward(data.x.float(),data.edge_index,data.batch)
        y = data.y.to(device)
        batch_size = data.batch_size
        target = torch.ones(batch_size)
        target = target.to(device)
        #loss = cos_ratio*self.loss_function1(outputs.to(torch.float32),y.view(batch_size,50).to(torch.float32),target)+mse_ratio*self.loss_function2(outputs.to(torch.float32),y.view(batch_size,50).to(torch.float32))
        loss = self.loss_function2(outputs.to(torch.float32),y.view(batch_size,50).to(torch.float32))
        self.progress.append(loss.item())        
        loss.backward()
        self.optimizer.step()
        
        return loss
        
    #def test(self,data):
        #outputs = self.forward(data.x.float(),data.edge_index,data.batch)
        #target = torch.ones(len(test_data))
        #target = target.to(device)
        #y = data.y.to(device)
        #acc = cos_ratio*self.loss_function1(outputs.to(torch.float32),y.view(len(test_data),50).to(torch.float32),target)+mse_ratio*self.loss_function2(outputs.to(torch.float32),y.view(len(test_data),50).to(torch.float32))
        #acc = self.loss_function2(outputs.to(torch.float32),y.view(batch_size,50).to(torch.float32))
        #return acc
    
    def pred(self,data):
        outputs = self.forward(data.x.float(),data.edge_index,data.batch)
        return outputs
    

hidden_channels=512
model_cos = GNN_COS()
model_mse = GNN_MSE()

model_mse.load_state_dict(torch.load("/home/chengc/workspace/cc/0815/model/qm_9_mse_model.pth"))
model_cos.load_state_dict(torch.load("/home/chengc/workspace/cc/0815/model/qm_9_cos_model.pth"))

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
model_mse.to(device)
model_cos.to(device)


In [ ]:
all_files = []
root_dir = "/home/chengc/workspace/cc/0815/data/cal_data/dude_graph"
for dirpath, dirnames, files in os.walk(root_dir):  
    for name in files:  
        all_files.append(os.path.join(dirpath, name)) 
def extract_labels(file_path):  
    # 分割文件名和目录部分  
    filename = os.path.basename(file_path)  
    # 去掉文件扩展名  
    filename_without_ext = os.path.splitext(filename)[0]  
    # 分割文件名以获得前缀和标签  
    parts = filename_without_ext.split('_')  
    # 假设前缀和标签分别位于倒数第二和最后一个位置  
    target_protein = parts[-2] if len(parts) > 1 else None  
    mode = parts[-1] if len(parts) > 0 else None  
    # 返回前缀和标签  
    return target_protein, mode


In [ ]:
for n in trange(len(all_files)):
    data = torch.load(all_files[n])
    data_loader = DataLoader(data,batch_size=1)
    input_data = []
    for loader in data_loader:
        input_data.append(loader)
    mse_predict = []
    for ii in range(len(input_data)):
        mse_predict.append(model_mse.pred(input_data[ii])[0].detach())
    
    cos_predict = []
    for iii in range(len(input_data)):
        cos_predict.append(model_cos.pred(input_data[iii])[0].detach())
    
    def datatran(m):
        mse_spec = mse_predict[m]
        cos_spec = cos_predict[m]*10000
        x100 = mse_spec.tolist()+cos_spec.tolist()
        x100 = torch.tensor(x100)
        receptor = input_data[m].target[0]
        smi = input_data[m].smi[0]
        mode = input_data[m].mode[0]
        if mode == "actives":
            y_c = torch.tensor(1)
        else : y_c =torch.tensor(0)
        return Data(x=x100,y=y_c,receptor=receptor,smi=smi)
    
    data100 = []
    targets,modes = extract_labels(all_files[n])
    save_dir = os.path.join("/home/chengc/workspace/cc/0815/data/cal_data/dude_spec_step1",targets+"_"+modes+".pt")
    for iiii in range(len(input_data)):
        point = datatran(iiii)
        data100.append(point)
    torch.save(data100,save_dir) 

In [ ]:
#step3 2in1

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
from tqdm import trange
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import torch.nn as nn
from torch.utils.data import random_split

In [ ]:
###load the model
hidden_channels = 256
class twoinone(nn.Module):
    
    def __init__(self):
        super().__init__()
        torch.manual_seed(42)
       
        self.nn1 = nn.Linear(100, hidden_channels*2)
        self.nn2 = nn.Linear(hidden_channels*2,hidden_channels*2)
        self.nn3 = nn.Linear(hidden_channels*2,hidden_channels*2)
        self.nn4 = nn.Linear(hidden_channels*2,hidden_channels*2)
        self.out = nn.Linear(hidden_channels*2, 50)
        
        self.loss_function = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.0001, weight_decay=5e-4)
        self.counter=0
        self.progress = []
        
    def forward(self,x):
        
        x = x.to(device)
        x=self.nn1(x)
        x = x.relu()
        x=self.nn2(x)
        x = x.relu()
        x=self.nn3(x)
        x = x.relu()
        x=self.nn4(x)
        x = x.relu()
        out = self.out(x)
        
        return out
    
    def train(self,data):
        loss_batch=[]
        for i in range(data.batch_size):            
            outputs = self.forward(data[i].x.float())
            y = data[i].y.to(device)
            y = y.float()
            loss_part = self.loss_function(outputs.to(torch.float32),y.to(torch.float32))
            loss_batch.append(loss_part)
        loss = torch.stack(loss_batch,dim=0).mean(dim=0)           
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss
    
    def val(self,data):
        vloss_batch=[]
        for i in range(data.batch_size):            
            outputs = self.forward(data[i].x.float())
            y = data[i].y.to(device)
            y = y.float()
            loss_part = self.loss_function(outputs.to(torch.float32),y.to(torch.float32))
            vloss_batch.append(loss_part)
        loss = torch.stack(vloss_batch,dim=0).mean(dim=0) 
        return loss
    
    def pred(self,data):
        output = self.forward(data.x)
        return output
    
device = torch.device('cuda:4' if torch.cuda.is_available() else 'cpu')
model = twoinone()
model.load_state_dict(torch.load("/home/chengc/workspace/cc/0815/model/qm_9_2in1_model.pth"))
model.to(device)

In [ ]:
all_files = []
root_dir = "/home/chengc/workspace/cc/0815/data/cal_data/dude_spec_step1"
for dirpath, dirnames, files in os.walk(root_dir):  
    for name in files:  
        all_files.append(os.path.join(dirpath, name)) 
def extract_labels(file_path):  
    # 分割文件名和目录部分  
    filename = os.path.basename(file_path)  
    # 去掉文件扩展名  
    filename_without_ext = os.path.splitext(filename)[0]  
    # 分割文件名以获得前缀和标签  
    parts = filename_without_ext.split('_')  
    # 假设前缀和标签分别位于倒数第二和最后一个位置  
    target_protein = parts[-2] if len(parts) > 1 else None  
    mode = parts[-1] if len(parts) > 0 else None  
    # 返回前缀和标签  
    return target_protein, mode

In [ ]:
for i in trange(len(all_files)):
    data = torch.load(all_files[i])
    targets,modes = extract_labels(all_files[i])
    save_dir = os.path.join("/home/chengc/workspace/cc/0815/data/cal_data/dude_spec_step2",targets+"_"+modes+".pt")
    for ii in trange(len(data)):
        data[ii].x = torch.tensor(model.pred(data[ii]).detach().tolist(),dtype = torch.float)
    torch.save(data,save_dir)

In [ ]:
###step4 +protein_spec

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
from tqdm import trange
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import torch.nn as nn
from torch.utils.data import random_split

#protein_max = 3.7739898962011713
#mol_max = 91.85801696777344

In [ ]:
pro_dir = "/home/chengc/workspace/cc/data/protein/dude"
def get_ir(receptor):
    ir_path = os.path.join(pro_dir,receptor+"_addelements_sig-IR-ML.txt")
    freq,ir = np.loadtxt(ir_path,usecols=(0,1), unpack=True)
    pro_y=[]
    for i in range(0,200,4):
        scale = max(ir)
        sub_sum = (sum(ir[i:i+4]) / scale) * 24
        pro_y.append(sub_sum)
    return pro_y

In [2]:
pro_82 = []
for dirpath, dirnames, files in os.walk('/home/chengc/workspace/cc/data/protein/dude'):  
    for name in files:  
        parts = name.split('_')
        pro_82.append(parts[0])

In [ ]:
all_files = []
root_dir = "/home/chengc/workspace/cc/data/dude/dude_spec_step2"
for dirpath, dirnames, files in os.walk(root_dir):  
    for name in files:  
        all_files.append(os.path.join(dirpath, name)) 
def extract_labels(file_path):  
    filename = os.path.basename(file_path)  
    filename_without_ext = os.path.splitext(filename)[0]  
    parts = filename_without_ext.split('_')  
    target_protein = parts[-2] if len(parts) > 1 else None  
    mode = parts[-1] if len(parts) > 0 else None   
    return target_protein, mode

In [ ]:
for i in trange(len(pro_82)):
    target = pro_82[i]
    a_data = torch.load(os.path.join("/home/chengc/workspace/cc/data/dude/dude_spec_step2",target+"_actives.pt"))
    d_data = torch.load(os.path.join("/home/chengc/workspace/cc/data/dude/dude_spec_step2",target+"_decoys.pt"))
    save_dir_a = os.path.join("/home/chengc/workspace/cc/0815/data/cal_data/dude_spec_withpro",target+"_actives.pt")
    save_dir_d = os.path.join("/home/chengc/workspace/cc/0815/data/cal_data/dude_spec_withpro",target+"_decoys.pt")
    pro_spec = get_ir(target)
    for point in a_data:
        point.x = torch.tensor(point.x.tolist()+pro_spec,dtype = torch.float)
    torch.save(a_data,save_dir_a)
    
    for dpoint in d_data:
        dpoint.x = torch.tensor(dpoint.x.tolist()+pro_spec,dtype = torch.float)
    torch.save(d_data,save_dir_d)

In [ ]:
#step5 -> train/val/test_data

In [4]:
root_dir = "/home/chengc/workspace/cc/0815/data/cal_data/dude_spec_withpro"
for i in trange(0,1):
    a_data = torch.load(os.path.join(root_dir,pro_82[i]+"_actives.pt"))
    d_data = torch.load(os.path.join(root_dir,pro_82[i]+"_decoys.pt"))
    atrain_num = int(len(a_data)*0.75)
    aval_num = int(len(a_data)*0.15)
    atest_num = len(a_data) - atrain_num - aval_num
    dtrain_num = atrain_num*10
    dval_num = aval_num*10
    dtest_num = len(d_data) - dtrain_num - dval_num               
    atrain,aval,atest = random_split(dataset = a_data,lengths = [atrain_num,aval_num,atest_num],generator = torch.Generator().manual_seed(42))
    dtrain,dval,dtest = random_split(dataset = d_data,lengths = [dtrain_num,dval_num,dtest_num],generator = torch.Generator().manual_seed(42))
    train_data = atrain+dtrain
    val_data = aval+dval
    test_data = atest+dtest
    
for i in trange(1,len(pro_82)):
    a_data = torch.load(os.path.join(root_dir,pro_82[i]+"_actives.pt"))
    d_data = torch.load(os.path.join(root_dir,pro_82[i]+"_decoys.pt"))
    atrain_num = int(len(a_data)*0.75)
    aval_num = int(len(a_data)*0.15)
    atest_num = len(a_data) - atrain_num - aval_num
    dtrain_num = atrain_num*10
    dval_num = aval_num*10
    dtest_num = len(d_data) - dtrain_num - dval_num               
    atrain,aval,atest = random_split(dataset = a_data,lengths = [atrain_num,aval_num,atest_num],generator = torch.Generator().manual_seed(42))
    dtrain,dval,dtest = random_split(dataset = d_data,lengths = [dtrain_num,dval_num,dtest_num],generator = torch.Generator().manual_seed(42))
    train_data = train_data+atrain+dtrain
    val_data = val_data+aval+dval
    test_data = test_data+atest+dtest
torch.save(train_data,'/home/chengc/workspace/cc/0815/data/cal_data/dude_82/train_data.pt')
torch.save(val_data,'/home/chengc/workspace/cc/0815/data/cal_data/dude_82/val_data.pt')
torch.save(test_data,'/home/chengc/workspace/cc/0815/data/cal_data/dude_82/test_data.pt')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 81/81 [03:48<00:00,  2.82s/it]
